In [5]:
import pandas as pd
import numpy as np

Untitled.ipynb	__init__.py  codebase.py  data	tests


In [54]:
dat = pd.read_csv('data/project_db_500.csv',low_memory=False)
dat.columns

Index(['script_ID', 'project_ID', 'sprite-type', 'sprite-name', 'script-rank',
       'coordinates', 'total-blocks', 'p_ID', 'project-name', 'username',
       'total-views', 'total-remixes', 'total-favorites', 'total-loves',
       'is-remix', 'Script_ID', 'block-rank', 'block-type', 'param1', 'param2',
       'param3', 'param4', 'param5', 'param6', 'param7', 'param8', 'param9',
       'param10', 'param11', 'param12', 'param13', 'param14', 'param15',
       'param16', 'param17', 'param18', 'param19', 'param20', 'param21',
       'param22', 'param23', 'param24'],
      dtype='object')

In [55]:
#replacing dashes with underscores
dat.columns = [i.replace('-', '_') for i in dat.columns]

dat = dat.drop(columns=['Script_ID','p_ID'])
dat = dat[dat.columns[~dat.columns.str.contains('param')]]

can certainly drop some of the columns, especially the redundant ones (and probably some others as well)
redundant: 
    script_ID and Script_ID
    project_ID and p_ID
will consider removing teh "param*" columns, since we're not really concerned with the pars put into the blocks

In [57]:
dat.project_ID.unique()

array([21194845, 32263244, 37655224, 61815506, 63182028, 66123158,
       67472712, 74061992, 75885516, 82328258, 88042639, 88148288,
       88349045, 89975999, 90425328, 93220839, 93353147, 93996328,
       94460384, 94528228, 94663417, 94747570, 95026293, 95348047,
       95350450, 95499588, 95828257, 96157479, 96194582, 96346900,
       96385817, 96436981, 96459421, 96590982, 96815253, 96950534,
       96968297, 97061132, 97180455, 97183802, 97269384, 97325861,
       97418828, 97443801, 97452769, 97492696, 97504731, 97539550,
       97550995, 97707762, 97729298, 97737168, 97896800, 97992888,
       98023098, 98030082, 98067195, 98138468, 98187031, 98200200,
       98233495, 98257615, 98262264, 98275228, 98329247, 98361543,
       98364564, 98372852, 98389189, 98429830, 98432921, 98438762,
       98462964, 98471445, 98474436, 98475360, 98481812, 98482554,
       98484602, 98489427, 98491513, 98494022, 98499667, 98500593,
       98504446, 98505630, 98506360, 98507494, 98510011, 98515